In [ ]:
# !pip install --upgrade google-cloud-aiplatform kfp
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
from kfp.v2.dsl import pipeline
from kfp.v2.dsl import component, InputPath, Model, OutputPath, Output, Metrics, Artifact, Dataset
from kfp.v2.dsl import pipeline
from kfp.v2.dsl import Input
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
@component(packages_to_install=["pandas", "numpy","fsspec", "gcsfs"])
def load_data(input_dataset_path:str, output_dataset: Output[Dataset]):
    import pandas as pd

    df = pd.read_csv(input_dataset_path,keep_default_na=False,na_values=[""])
    df.to_csv(output_dataset.path, index = False)


In [ ]:
@component(packages_to_install=["pandas", "scikit-learn"])
def split_data(input_data_path: InputPath('Dataset'),
               training_data_path: OutputPath('Dataset'),
               validation_data_path: OutputPath('Dataset')):
    import pandas as pd
    from sklearn.model_selection import train_test_split

    # Load the dataset
    df = pd.read_csv(input_data_path,keep_default_na=False,na_values=[""])

    # Splitting the dataset into training and validation sets
    # Assuming the last column is the target
    features = df.iloc[:, :-1]
    target = df.iloc[:, -1]
    X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.3, random_state=42)

    # Combining features and targets for training and validation sets
    train_df = pd.concat([X_train, y_train], axis=1)
    val_df = pd.concat([X_val, y_val], axis=1)

    # Saving the split datasets to the respective output paths
    train_df.to_csv(training_data_path, index=False)
    val_df.to_csv(validation_data_path, index=False)


In [ ]:
@component(packages_to_install=["pandas", 'numpy'])
def preprocess_data(input_data_path: InputPath('Dataset'),
                    output_data_path: OutputPath('Dataset')):
    import pandas as pd
    import numpy as np

    df = pd.read_csv(input_data_path,keep_default_na=False,na_values=[""])

    # 1. Handling Missing Values
    df['Number of Prior Visits'] = df['Number of Prior Visits'].fillna(df['Number of Prior Visits'].mode()[0])
    df['Medications Prescribed'] = df['Medications Prescribed'].fillna(df['Medications Prescribed'].mode()[0])

    # Remove age outliers
    df = df[df['Age'] <= 100]

    # 2. Feature Engineering
    exercise_map = {'None': 0, 'Occasional': 1, 'Regular': 2}
    df['Exercise_Encoded'] = df['Exercise Frequency'].map(exercise_map)

    def bmi_category(bmi):
        if bmi < 18.5:
            return 'Underweight'
        elif bmi < 25:
            return 'Normal'
        elif bmi < 30:
            return 'Overweight'
        else:
            return 'Obese'
    df['BMI_Category'] = df['BMI'].apply(bmi_category)

    def age_group(age):
        if age < 40:
            return '<40'
        elif age < 65:
            return '40-64'
        else:
            return '65+'
    df['Age_Group'] = df['Age'].apply(age_group)

    # One-hot encode
    df = pd.get_dummies(df, columns=[
        'Gender', 'Ethnicity', 'Diet Type', 'Type of Treatment',
        'BMI_Category', 'Age_Group'
    ], drop_first=True)

    # 3. skewed variable
    df['LOS_Log'] = np.log1p(df['Length of Stay'])


    # 4. Feature Selection
    df = df.drop(columns=[
        'Hospital ID', 'Adjusted Weight (kg)', 'Weight (kg)', 'Exercise Frequency', 'Length of Stay'
    ])

    df.to_csv(output_data_path, index=False)


In [ ]:
@component(packages_to_install=["pandas", "scikit-learn"])
def normalize_training_data(dataset_path: InputPath('Dataset'),
                            normalized_dataset_path: OutputPath('Dataset'),
                            scaler_artifact: OutputPath('Artifact')):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import joblib

    # Load the dataset
    df = pd.read_csv(dataset_path)
    X = df.drop(columns=['PatientID', 'Readmission within 30 Days'])
    target = df['Readmission within 30 Days']

    # Normalize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(X)

    # Save the scaler to the output path
    joblib.dump(scaler, scaler_artifact)

    # Combine scaled features with the target into a new DataFrame
    df_scaled = pd.DataFrame(features_scaled, columns=X.columns)
    df_scaled['Readmission within 30 Days'] = target  # Reattach the target column

    # Save the normalized dataset
    df_scaled.to_csv(normalized_dataset_path, index=False)

In [ ]:
@component(packages_to_install=["pandas", "scikit-learn"])
def normalize_validation_data(dataset_path: InputPath('Dataset'),
                              normalized_dataset_path: OutputPath('Dataset'),
                              scaler_artifact: InputPath('Artifact')):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import joblib

    # Load the dataset
    df = pd.read_csv(dataset_path)
    X = df.drop(columns=['PatientID', 'Readmission within 30 Days'])
    target = df['Readmission within 30 Days']

    # Normalize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(X)

    # Save the scaler to the output path
    joblib.dump(scaler, scaler_artifact)

    # Combine scaled features with the target into a new DataFrame
    df_scaled = pd.DataFrame(features_scaled, columns=X.columns)
    df_scaled['Readmission within 30 Days'] = target  # Reattach the target column

    # Save the normalized dataset
    df_scaled.to_csv(normalized_dataset_path, index=False)

In [ ]:
@component(packages_to_install=["pandas", "imblearn"])
def smote_augment_data(dataset_path: InputPath('Dataset'),
                       output_dataset_path: OutputPath('Dataset')):
    import pandas as pd
    from imblearn.over_sampling import SMOTE

    # Load normalized dataset
    df = pd.read_csv(dataset_path)

    # Split features and target
    X = df.drop(columns=['Readmission within 30 Days'])
    y = df['Readmission within 30 Days']

    # Apply SMOTE
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)

    # Combine back into a DataFrame
    df_resampled = pd.DataFrame(X_res, columns=X.columns)
    df_resampled['Readmission within 30 Days'] = y_res

    # Save the new dataset
    df_resampled.to_csv(output_dataset_path, index=False)


In [ ]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib"])
def train_model(training_data_path: InputPath('Dataset'), output_model: Output[Model]):
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    import joblib

    # Load the combined dataset
    df = pd.read_csv(training_data_path)
    # Assuming the last column is the target
    X_train = df.iloc[:, :-1]
    y_train = df.iloc[:, -1]

    # Create and train the model
    model = LogisticRegression()
    model.fit(X_train, y_train.values.ravel())

    # Save the trained model to a file
    joblib.dump(model, output_model.path)

In [ ]:
@component(packages_to_install=["pandas", "scikit-learn", "joblib"])
def evaluate_model(model: Input[Model], validation_data_path: InputPath('Dataset'), metrics: Output[Metrics]):
    import pandas as pd
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score

    import joblib

    # Load
    model = joblib.load(model.path)
    val_data = pd.read_csv(validation_data_path)

    # Selecting features and target using iloc
    X_val_scaled = val_data.iloc[:, :-1]  # All columns except the last one as features
    y_val = val_data.iloc[:, -1]  # Last column as target

    # Predictions
    y_val_pred = model.predict(X_val_scaled)
    y_val_pred_proba = model.predict_proba(X_val_scaled)[:, 1]  # Probability of the positive class

    # Calculating metrics
    accuracy = accuracy_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    class_report = classification_report(y_val, y_val_pred, output_dict=True)
    f1 = f1_score(y_val, y_val_pred)
    auc = roc_auc_score(y_val, y_val_pred_proba)

    # Log the evaluation metrics
    metrics.log_metric("accuracy", accuracy)
    metrics.log_metric("confusion_matrix", conf_matrix.tolist())
    metrics.log_metric("classification_report", class_report)
    metrics.log_metric("f1_score", f1)
    metrics.log_metric("auc", auc)

In [ ]:
@pipeline(name='healthcare_readmissions_training_pipeline')
def healthcare_readmissions_training_pipeline(healthcare_readmissions_dataset_path:str):

    load_data_task = load_data(input_dataset_path = healthcare_readmissions_dataset_path)

    preprocess_task = preprocess_data(
        input_data_path=load_data_task.output
    )

    split_result = split_data(input_data_path=preprocess_task.output)

    normalize_training = normalize_training_data(
        dataset_path=split_result.outputs['training_data_path']
    )

    # ➕ Apply SMOTE
    smote_task = smote_augment_data(
        dataset_path=normalize_training.outputs['normalized_dataset_path']
    )

    normalize_validation = normalize_validation_data(
        dataset_path=split_result.outputs['validation_data_path'],
        scaler_artifact=normalize_training.outputs['scaler_artifact']
    )

    trained_model = train_model(training_data_path=smote_task.output)

    evaluate_task = evaluate_model(
        model=trained_model.output,
        validation_data_path=normalize_validation.outputs['normalized_dataset_path']
    )


In [ ]:
from kfp.v2 import compiler

# Compile the pipeline
compiler.Compiler().compile(
    pipeline_func=healthcare_readmissions_training_pipeline,
    package_path='healthcare_readmissions_training_pipeline.json'  # This is the output file
)

In [ ]:
from google.cloud import aiplatform

pipeline_job = aiplatform.PipelineJob(
    display_name='healthcare_readmissions_training_pipeline',
    template_path='healthcare_readmissions_training_pipeline.json',  # Updated to the correct pipeline file name
    pipeline_root=healthcare_readmissions_dataset,
    parameter_values={
      'healthcare_readmissions_dataset_path': f'{healthcare_readmissions_dataset}/healthcare_readmissions_dataset_train.csv'
    },
    enable_caching=True
)

In [ ]:
pipeline_job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/652395460584/locations/us-central1/pipelineJobs/healthcare-readmissions-training-pipeline-20250506232547
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/652395460584/locations/us-central1/pipelineJobs/healthcare-readmissions-training-pipeline-20250506232547')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/healthcare-readmissions-training-pipeline-20250506232547?project=652395460584
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/652395460584/locations/us-central1/pipelineJobs/healthcare-readmissions-training-pipeline-20250506232547 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.c